In [1]:
import drivers
import params
import vumps
import matrices
import numpy as np
import contractions as ct
import time
import tensornetwork as tn
import tensornetwork.linalg.operations
import jax
import jax.config
jax.config.update("jax_enable_x64", True)
%load_ext autoreload
%autoreload 2

In [2]:
chi = 16
vumps_params = params.vumps_params()
heff_params = params.krylov_params(n_krylov=40, max_restarts=10)
env_params = params.gmres_params(n_krylov=40, max_restarts=10)
data, timing = drivers.vumps_XX(chi, vumps_params=vumps_params, heff_params=heff_params, env_params=env_params, 
                                dtype=np.float32, backend="jax")
print(timing)

/Users/adam/opt/anaconda3/lib/python3.7/site-packages/jax/lib/xla_bridge.py:116: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


vuMPS, a love story.
**************************************************************
Initial solve time: 1.5790006950000004
Initial energy: 0.021150038
And so it begins...
N = 1| eps = 5.06e-01| E = 1.0057337284088135| dE = 9.85e-01| dt= 3.93e+00
N = 2| eps = 4.91e-01| E = 1.0316145420074463| dE = 2.59e-02| dt= 1.12e+00
N = 3| eps = 3.64e-01| E = 1.3690930604934692| dE = 3.37e-01| dt= 1.16e+00
N = 4| eps = 9.09e-02| E = 1.2606631517410278| dE = 1.08e-01| dt= 1.10e+00
N = 5| eps = 6.57e-03| E = 1.2704749107360840| dE = 9.81e-03| dt= 1.10e+00
N = 6| eps = 2.78e-03| E = 1.2715026140213013| dE = 1.03e-03| dt= 1.09e+00
N = 7| eps = 1.57e-03| E = 1.2720712423324585| dE = 5.69e-04| dt= 1.11e+00
N = 8| eps = 9.65e-04| E = 1.2723708152770996| dE = 3.00e-04| dt= 1.10e+00
Convergence achieved at iteration 8
The main loops took 11.879987503999999 seconds.
Simulation finished. Pickling results.
Pickling to ./vumps/pickles/_t8.pkl
{'Iteration': 1.0997909719999992, 'Gradient': 0.022143824999998785, 'H

In [3]:
chi = 16
vumps_params = params.vumps_params()
heff_params = params.krylov_params(n_krylov=40, max_restarts=10)
env_params = params.gmres_params(n_krylov=40, max_restarts=10)
data, timing = drivers.vumps_XX(chi, vumps_params=vumps_params, heff_params=heff_params, env_params=env_params, 
                                dtype=np.float32, backend="numpy")
print(timing)

vuMPS, a love story.
**************************************************************
Initial solve time: 0.025835689000000883
Initial energy: 0.027315743
And so it begins...
N = 1| eps = 1.35e+00| E = 0.6674837470054626| dE = 6.40e-01| dt= 1.01e-01
N = 2| eps = 5.39e-01| E = 1.2345227003097534| dE = 5.67e-01| dt= 9.25e-02
N = 3| eps = 6.97e-01| E = 1.1996443271636963| dE = 3.49e-02| dt= 9.28e-02
N = 4| eps = 8.01e-01| E = 1.1897722482681274| dE = 9.87e-03| dt= 9.49e-02
N = 5| eps = 4.35e-01| E = 1.0220650434494019| dE = 1.68e-01| dt= 9.72e-02
N = 6| eps = 8.99e-02| E = 1.2636640071868896| dE = 2.42e-01| dt= 9.46e-02
N = 7| eps = 7.17e-03| E = 1.2690889835357666| dE = 5.42e-03| dt= 7.44e-02
N = 8| eps = 4.25e-03| E = 1.2711669206619263| dE = 2.08e-03| dt= 7.72e-02
N = 9| eps = 3.04e-03| E = 1.2718585729598999| dE = 6.92e-04| dt= 6.04e-02
N = 10| eps = 1.87e-03| E = 1.2722179889678955| dE = 3.59e-04| dt= 5.48e-02
N = 11| eps = 1.05e-03| E = 1.2724289894104004| dE = 2.11e-04| dt= 5.46e-02


# Jax vs Numpy Timings

# ncon

In [77]:
chi = 256
A = tn.randn((chi, chi), dtype=np.float64, backend="jax")
B = tn.randn((chi, chi), dtype=np.float64, backend="jax")
C = tn.linalg.operations.ncon([A, B], [[-1, 2], [2, -2]])
dummy = C.array.block_until_ready()

time0 = time.time()
C = tn.linalg.operations.ncon([A, B], [[-1, 2], [2, -2]])
dummy = C.array.block_until_ready()
time1 = time.time()
dt = time1 - time0
print("T tn:", dt)

time0 = time.time()
C = tn.ncon([A.array, B.array], [[-1, 2], [2, -2]], backend="jax")
dummy = C.block_until_ready() 
time1 = time.time()
dt = time1 - time0
print("T old:", dt)

T tn: 0.0005581378936767578
T old: 0.0006101131439208984


# leftmult

In [103]:
chi = 256
d = 2
A = tn.randn((chi, chi), dtype=np.float64, backend="jax")
B = tn.randn((chi, d, chi), dtype=np.float64, backend="jax")

time0 = time.time()
C = ct.leftmult(A, B)
dummy = C.array.block_until_ready()
time1 = time.time()
dt = time1 - time0
print("T jax run1:", dt)

time0 = time.time()
C = ct.leftmult(A, B)
dummy = C.array.block_until_ready()
time1 = time.time()
dt = time1 - time0
print("T jax run2:", dt)

A = tn.randn((chi, chi), dtype=np.float64, backend="numpy")
B = tn.randn((chi, d, chi), dtype=np.float64, backend="numpy")

time0 = time.time()
C = ct.leftmult(A, B)
time1 = time.time()
dt = time1 - time0
print("T numpy:", dt)

T jax run1: 0.0075190067291259766
T jax run2: 0.0009410381317138672
T numpy: 0.0008330345153808594


# XopL

In [108]:
chi = 256
d = 2
A = tn.randn((chi, chi), dtype=np.float64, backend="jax")
B = tn.randn((chi, d, chi), dtype=np.float64, backend="jax")

time0 = time.time()
C = ct.XopL(B, A)
dummy = C.array.block_until_ready()
time1 = time.time()
dt = time1 - time0
print("T jax run1:", dt)

time0 = time.time()
C = ct.XopL(B, A)
dummy = C.array.block_until_ready()
time1 = time.time()
dt = time1 - time0
print("T jax run2:", dt)

A = tn.randn((chi, chi), dtype=np.float64, backend="numpy")
B = tn.randn((chi, d, chi), dtype=np.float64, backend="numpy")

time0 = time.time()
C = ct.XopL(B, A)
time1 = time.time()
dt = time1 - time0
print("T numpy:", dt)

T jax run1: 0.0015943050384521484
T jax run2: 0.0018112659454345703
T numpy: 0.0017979145050048828


# apply Hc

In [14]:
chi = 32
d = 2
dtype = np.float64
A_L, C, A_R, H = random_hermitian_system("jax", dtype, chi, d)
LH = random_hermitian_matrix("jax", dtype, chi)
RH = random_hermitian_matrix("jax", dtype, chi)

time0 = time.time()
newC = ct.apply_Hc(C, A_L, A_R, [H, LH, RH])
dummy = newC.array.block_until_ready()
time1 = time.time()
dt = time1 - time0
print("T jax run1:", dt)

time0 = time.time()
newC = ct.apply_Hc(C, A_L, A_R, [H, LH, RH])
dummy = newC.array.block_until_ready()
time1 = time.time()
dt = time1 - time0
print("T jax run2:", dt)

A_L, C, A_R, H = random_hermitian_system("numpy", dtype, chi, d)
LH = random_hermitian_matrix("numpy", dtype, chi)
RH = random_hermitian_matrix("numpy", dtype, chi)
time0 = time.time()
newC = ct.apply_Hc(C, A_L, A_R, [H, LH, RH])
time1 = time.time()
dt = time1 - time0
print("T numpy run1:", dt)

T jax run1: 0.07713603973388672
T jax run2: 0.0018491744995117188
T numpy run1: 0.0010991096496582031


# Hc matvec

In [20]:
chi = 256
d = 2
dtype = np.float64
A_L, C, A_R, H = random_hermitian_system("jax", dtype, chi, d)
LH = random_hermitian_matrix("jax", dtype, chi)
RH = random_hermitian_matrix("jax", dtype, chi)

time0 = time.time()
newC = vumps.Hc_matvec(C.array, A_L.array, A_R.array, H.array, LH.array, RH.array, "jax")
dummy = newC.block_until_ready()
time1 = time.time()
dt = time1 - time0
print("T jax run1:", dt)

time0 = time.time()
newC = vumps.Hc_matvec(C.array, A_L.array, A_R.array, H.array, LH.array, RH.array, "jax")
dummy = newC.block_until_ready()
time1 = time.time()
dt = time1 - time0
print("T jax run2:", dt)

A_L, C, A_R, H = random_hermitian_system("numpy", dtype, chi, d)
LH = random_hermitian_matrix("numpy", dtype, chi)
RH = random_hermitian_matrix("numpy", dtype, chi)
time0 = time.time()
newC = vumps.Hc_matvec(C.array, A_L.array, A_R.array, H.array, LH.array, RH.array, "numpy")
time1 = time.time()
dt = time1 - time0
print("T numpy run1:", dt)

T jax run1: 0.055910348892211914
T jax run2: 0.008743762969970703
T numpy run1: 0.011023998260498047


# Minimize Hc

In [2]:
def random_hermitian_system(backend, dtype, chi, d):
  """
  Return A_L, C, A_R representing a normalized quantum state and a Hermitian
  H.
  """
  A_1 = tn.randn((chi, d, chi), dtype=dtype, backend=backend, seed=10)
  A_L, _ = tn.linalg.linalg.qr(A_1, pivot_axis=2, non_negative_diagonal=True)
  C, A_R = tn.linalg.linalg.rq(A_L, pivot_axis=1, non_negative_diagonal=True)
  C /= tn.linalg.linalg.norm(C)
  H = tn.randn((d, d, d, d), dtype=dtype, backend=backend, seed=10)
  H = H.reshape((d*d, d*d))
  H = 0.5*(H + H.H)
  H = H.reshape((d, d, d, d))
  return (A_L, C, A_R, H)

def random_hermitian_matrix(backend, dtype, chi):
  A = tn.randn((chi, chi), dtype=dtype, backend=backend, seed=10)
  return 0.5*(A + A.H)

In [34]:
chi = 128
d = 2
dtype = np.float64
A_L, C, A_R, H = random_hermitian_system("jax", dtype, chi, d)
A_C = ct.leftmult(C, A_R)
LH = random_hermitian_matrix("jax", dtype, chi)
RH = random_hermitian_matrix("jax", dtype, chi)
the_params = params.krylov_params(n_krylov=chi, max_restarts=5)

time0 = time.time()
_, newC = vumps.minimize_HAc([A_L, C, A_R], A_C, [H, LH, RH], 0.1, the_params)
dummy = newC.array.block_until_ready()
time1 = time.time()
dt = time1 - time0
print("T jax run1:", dt)

time0 = time.time()
_, newC = vumps.minimize_HAc([A_L, C, A_R], A_C, [H, LH, RH], 0.1, the_params)
dummy = newC.array.block_until_ready()
time1 = time.time()
dt = time1 - time0
print("T jax run2:", dt)

A_L, C, A_R, H = random_hermitian_system("numpy", dtype, chi, d)
A_C = ct.leftmult(C, A_R)
LH = random_hermitian_matrix("numpy", dtype, chi)
RH = random_hermitian_matrix("numpy", dtype, chi)
time0 = time.time()
_, newC = vumps.minimize_HAc([A_L, C, A_R], A_C, [H, LH, RH], 0.1, the_params)
time1 = time.time()
dt = time1 - time0
print("T numpy run1:", dt)

T jax run1: 0.8778889179229736
T jax run2: 0.8458349704742432
T numpy run1: 0.3234670162200928


In [ ]:
D=chi*chi*d
dtype=np.float64
matrix = jax.numpy.array(np.random.rand(D,D).astype(dtype))
vector = jax.numpy.array(np.random.rand(D,).astype(dtype))
@jax.jit
def matvec_jax_matrix(vec,matrix):
    return jax.numpy.tensordot(matrix, vec,([1],[0]))
jax_backend = tn.backends.jax.jax_backend.JaxBackend()
ncv=10
t1 = time.time()
eta_j, U_j = jax_backend.eigsh_lanczos(matvec_jax_matrix,[matrix],vector,num_krylov_vecs = ncv,numeig=1, 
                                       reorthogonalize=False)
print('jax eigvals:', eta_j)
t2 = time.time()
eta_j, U_j = jax_backend.eigsh_lanczos(matvec_jax_matrix,[matrix],vector,num_krylov_vecs = ncv,numeig=1, 
                                       reorthogonalize=False)

print('jax eigvals:', eta_j)
t3 = time.time()
print('jax first:', t2 - t1)
print('jax second:', t3 - t2)